In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

path='/Users/brianna/Documents/WL_DBdeets/'

In [ ]:
# Read in the dataframe that has retention information for each user.
retention = pd.read_csv(path+'RetentionTemp.csv')
print(retention.columns)
print(len(retention.index))


# Read in the dataframe with meetings data for one of the weeks
# This needs to get fixed later to refer to meetings at Week0
meetings = pd.read_csv(path+'2016-01-10 00:00:00to2016-01-17 00:00:00EvntsCollapsedByUser.csv')
print(meetings.columns)
print(len(meetings.index))


df = pd.DataFrame.merge(retention, meetings, on='user_id', how='outer')

df[df.MeetingsTotal.isnull()]=0

In [ ]:
df.head(5)

In [ ]:
# Plot number of meetings in a given week
plt.figure(figsize=(20,10))
n, bins, patches = plt.hist(df.MeetingsTotal[df.MeetingsTotal<100], 50, facecolor='green', alpha=0.75)

plt.xlabel('Number of Meetings Per Week (Week of 2016-01-10)', size=20)

In [ ]:
# Make a function to calculate percentage of users active in each week
def calculateStats(df):
    # Plot the retention curve of all of the people.
    # Find the number of people who went to the website on WeekX and 
    # divide by the total number of entries in WeekX.
    # (You can't just divide by total number of entries because people that
    # joined fewer than 8 weeks ago don't have entries for all of the weeks.)
    sumDF=df.sum()

    # Make a tiny dataframe with a row for every week (0,1,2,etc)
    # And columns for numActive, totalActive, and percentActive
    retenStats = pd.DataFrame(columns=['Week','numActive', 'numTotal','percentActive'])

    for x in range(8):  
        Week = x
        numActive = sumDF['VisitedWeek'+str(x)]
        numTotal = df['VisitedWeek'+str(x)].count()
        percentActive = numActive/numTotal
        retenStats.loc[x] = [Week,numActive,numTotal,percentActive]

    return retenStats
    #percentActive=numActive/numTotal

In [ ]:
all = calculateStats(df)
meet0 = calculateStats(df[df.MeetingsTotal==0])
meet1to4 = calculateStats(df[(df.MeetingsTotal>0) & (df.MeetingsTotal<5)])
meet5to8 = calculateStats(df[(df.MeetingsTotal>4) & (df.MeetingsTotal<9)])
meet9to12 = calculateStats(df[(df.MeetingsTotal>8) & (df.MeetingsTotal<12)])
meet12plus = calculateStats(df[df.MeetingsTotal>12])

In [ ]:
print(all.head())
print(meet0.head())
print(meet1to4.head())

In [ ]:
plt.figure(figsize=(20,10))

#plt.plot(all.Week, all.percentActive, label="All Users")
plt.plot(meet0.Week, meet0.percentActive, label="0 Meetings/Wk")
plt.plot(meet1to4.Week, meet1to4.percentActive, label="1-4 Meetings/Wk")
plt.plot(meet5to8.Week, meet5to8.percentActive, label="5-8 Meetings/Wk")
plt.plot(meet9to12.Week, meet9to12.percentActive, label="9-12 Meetings/Wk")
plt.plot(meet12plus.Week, meet12plus.percentActive, label="12+ Meetings/Wk")

plt.ylim([0, 1])

plt.xlabel('Weeks from Sign-up', size=20)
plt.ylabel('Percentage of Users', size=20)
plt.legend(loc = 'upper center', fontsize=20)
# plt.legend(')